In [37]:
## Import all the necessary packages

import random

from googlesearch import search 

import pandas as pd

import requests

from bs4 import BeautifulSoup

import re


In [ ]:
## RUN ONLY THE FIRST TIME

In [40]:
## Scrape Wikipedia to create a library of themes 

# Get the url of the webpage contening the list of macro categories under Human A 
macro_url = requests.get('https://en.wikipedia.org/w/index.php?title=Portal:Contents/Categories&action=info').text

# Process the content of the page
macro_soup = BeautifulSoup(macro_url,'lxml')

# Create an empty list to store all categories
macro_category = []

# All of the macro categories were in a '<li>' brackets
for i in macro_soup.find_all(name = 'li'):
    # pull the actual category for each one 
    for link in i.find_all('a', href=True):
        macro_category.append(link['href'])

# Selecting only the category names
macro_category = [re.sub('/wiki/Portal:Contents/Categories/', '', cat) for cat in macro_category if '/wiki/Portal' in str(cat)]

print('We have: {} categories'.format(len(macro_category)))
print('Example of categories: ', macro_category[:5])

We have: 25 categories
Example of categories:  ['Culture_and_the_arts', 'Geography_and_places', 'Health_and_fitness', 'History_and_events', 'Human_activities']


In [69]:
# Using the macro categories just compiled to scrape the subcategories on each individual page

# Create an empty list to store all categories
full_category = []

# Create a loop to scrape each individual macro category page for subcategories  
for cat in macro_category:
    micro_url = requests.get('https://en.wikipedia.org/wiki/Portal:Contents/Categories/' + str(cat)).text
    micro_soup = BeautifulSoup(micro_url,'lxml')
    
    micro_category = []
    for i in micro_soup.find_all(name = 'li'):
        for link in i.find_all('a', href=True):
            micro_category.append(link['href'])
            
    # Selecting only the category names
    df = [cat.split(':')[1] for cat in micro_category if cat.split(':')[0] == '/wiki/Category']
        
    full_category.append(df)
    
# Create the final dataframe

# Create an empty list to store micro and macro category pairs
categories = []
for i in range(len(full_category)):
    for j in range(len(full_category[i])):
        # Compiling the pairs and cleaning the text
        cat = list([full_category[i][j].replace('_', ' '), macro_category[i].replace('_', ' ')])
        categories.append(cat)

# Create a dataframe and drop eventual duplicates
themes_df = pd.DataFrame(themes, columns=['categories', 'themes'])
themes_df = themes_df.drop_duplicates()

# Print some information
print('We have {} different categories from {} themes'.format(themes_df.shape[0], len(macro_category)))

themes_df.head()

We have 741 different categories from 25 themes


,categories,themes
0,Culture,Culture and the arts
1,Humanities,Culture and the arts
2,Classical studies,Culture and the arts
3,Critical theory,Culture and the arts
4,Cultural anthropology,Culture and the arts


In [70]:
# Save the dataframe for later use
themes_df.to_csv('D:\Projects\Mind_Expander\data_themes.csv')

In [68]:
# Create a feedback dataframe for later use
general_history = pd.DataFrame(columns=['date','article', 'grade', 'category', 'themes'])
general_history.to_csv('D:\Projects\Mind_Expander\data_feedback.csv')